# Проект приоритизация гипотез. Анализ A/B-теста.

В данном проекте предстоит проанализировать датасет интернет-магазина. Совместно с отделом маркетинга подготовлен список гипотез для увеличения выручки.

В файле 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort.

Задача проекта исследовать результаты А/В теста и принять нужное решение по результатам:
1. Остановить тест, зафиксировать победу одной из групп. 
2. Остановить тест, зафиксировать отсутствие различий между группами. 
3. Продолжить тест.

##  Исследование данных.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import math
from scipy import stats

from datetime import timedelta
import datetime as dt

from statsmodels.stats.proportion import proportions_ztest
import warnings

warnings.filterwarnings('ignore')

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
hypothesis = pd.read_csv('/datasets/hypothesis.csv', sep=',')

In [ ]:
hypothesis.columns = [x.lower() for x in hypothesis.columns.values]
display(hypothesis)

###  Фреймворк ICE

In [ ]:
hypothesis['ICE'] = (hypothesis['impact']*hypothesis['confidence']/hypothesis['efforts']).round(2)
pd.set_option('display.max_colwidth', 500)
hypothesis[['hypothesis', 'ICE']].sort_values(by='ICE',ascending=False)

###  Фреймворк RICE

In [ ]:
hypothesis['RICE'] =(hypothesis['reach']*hypothesis['impact']*hypothesis['confidence'])/hypothesis['efforts']
pd.set_option('display.max_colwidth', 500)
hypothesis[['hypothesis', 'RICE','reach']].sort_values(by='RICE', ascending=False)

In [ ]:
hypothesis[['hypothesis', 'ICE', 'RICE']]

Приоритеты по гипотезам ICE и RICE отличаются,так как количество пользователей, которых она затрагивает в Reach (скольких пользователей затронет изменение, которое мы хотим внести), разное. В связи с этим есть смысл в первую очередь рассмотреть гипотезы 7 и 2.

## Анализ A/B-теста. Предобработка данных.

Посмотрим результаты А/В тестов

In [ ]:
visitors = pd.read_csv('/datasets/visitors.csv', sep=',')
orders = pd.read_csv('/datasets/orders.csv', sep=',')

In [ ]:
visitors.info()

In [ ]:
visitors.head()

Переведём столбец date в формат datetime

In [ ]:
visitors['date'] = visitors['date'].map(
      lambda x: dt.datetime.strptime(x, '%Y-%m-%d')
)

In [ ]:
visitors['visitors'].describe()

В таблице visitors 62 строки,среднее и медиана близки по цифрам (607 и 624 соответственно).

In [ ]:
visitors['date'].describe()

В группах А и В 31 уникальных значений в каждой группе, данные предоставлены за август 2019 года.

In [ ]:
visitors['group'].value_counts()

Пользователей в группа А и В равное количество-31.

In [ ]:
display(visitors.duplicated().sum())

Дубликатов нет.  

Посмотрим на данные в таблице orders.

In [ ]:
orders.info()

In [ ]:
orders.head()

Так же переведем столбец в datetime, а так же перименуем столбцы transactionId и visitorId

In [ ]:
orders['date'] = orders['date'].map(
    lambda x: dt.datetime.strptime(x, '%Y-%m-%d')
)

In [ ]:
orders = orders.rename(columns={"transactionId": "orderId", "visitorId": "userId"})

Посмторим данные по столбцам

In [ ]:
orders["date"].describe()

В таблице заказы за август 2019 года. Датасет содержит 1197 строк, из них уникальных - 31. Больше всего заказов (48) сделано 2019-08-05.

In [ ]:
orders['userId'].value_counts()

In [ ]:
display(orders.duplicated().sum())

In [ ]:
orders_visitors = orders.pivot_table(index='userId', values='group', aggfunc='nunique')
orders_visitors.columns = ['count', 'columns']
orders_visitors.reset_index()
orders_visitors.query('count > 1').count()

In [ ]:
duplicates_users = np.intersect1d(orders.query('group == "A"')['userId'].unique(), 
                   orders.query('group == "B"')['userId'].unique())
orders[~orders['userId'].isin(duplicates_users)]
display(duplicates_users)

58 пользователей оказались в обеих группах. 
Данные готовы к анализу, дубликатов нет, тип столбцов date изменили на datetime.

ОБЩИЙ ИТОГ:
Таблица с гипотезами оставлена без изменений,так как данные корректны. В датафреймах с заказами и посетителями изменен тип столбца с датой. Диапазон (август 2019) дат в таблицах заказов и посетителей совпадают.

### График кумулятивной выручки по группам

In [ ]:
# создаем массив уникальных пар значений дат и групп теста
datesGroups = orders[['date','group']].drop_duplicates()

# получаем агрегированные кумулятивные по дням данные о заказах 
ordersAggregated = datesGroups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'orderId' : 'nunique', 'userId' : 'nunique', 'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])

# получаем агрегированные кумулятивные по дням данные о посетителях интернет-магазина 
visitorsAggregated = datesGroups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])

# объединяем кумулятивные данные в одной таблице и присваиваем ее столбцам понятные названия
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

print(cumulativeData.head(5))

In [ ]:
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

plt.figure(figsize=(15,5))
# график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')
# график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')

plt.legend()
plt.grid()
plt.title('Кумулятивная выручка по группам')
plt.xlabel('Дата, день')
plt.ylabel('Выручка, руб')
plt.show()

Выручка в обеих группах равномерно увеличиваетсся, однако группа В лидирует, после 17.08.2019 года она резко вырастает, возможно заказы стали дороже или их стало больше. Нужно посмотреть, что именно произошло.   

Посмотрим на заказы с 1.08 по 21.08:

In [ ]:
cumulativeRevenueB.query(' "2019-08-17" <= date <= "2019-08-21" ')

Видим, что количество заказов в пределах одних цифр, то есть количество сильно не менялось.

In [ ]:
display('Медианное значение заказов 19.08.2019 =',orders.query(' date == "2019-08-19" ')['revenue'].median())
orders.query(' date == "2019-08-19" ').sort_values(by='revenue', ascending=False).head(8)

Видим,что заказ с номером 425 явно больше остальных сумм -1294500 рублей против 17000-44000 рублей. То есть можно сказать,что рост на графике продиктован случайностью.

###  График кумулятивного среднего чека по группам. 

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue']/cumulativeRevenueA['orders'], label='A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue']/cumulativeRevenueB['orders'], label='B')

plt.grid()
plt.title('Кумулятивный средний чек по группам')
plt.xlabel('Дата, день')
plt.ylabel('Средний чек, руб')
plt.legend() 
plt.show()

Есть схожесть с графиком кум.выручки. В первой половине в группе "B" чек преимущественно больше, но есть даты где и наоборот, чек больше в "A" , например,  начало 1 августа и 12-14 августа. Потом чек группы "B" стабильно больше, вмесе с тем на графике видно сильное влияние на средний чек заказа 425, но после него (19 августа) средний чек в "B" постепенно пошел вниз, в то время как в "A" он с 18 августа постепенно растет.

###  График относительного изменения кумулятивного среднего чека группы B к группе A.

In [ ]:
# собираем данные в одном датафрейме
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

plt.figure(figsize=(15,5))
plt.grid()
# cтроим отношение средних чеков
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['revenueB']/mergedCumulativeRevenue['ordersB'])/(mergedCumulativeRevenue['revenueA']/mergedCumulativeRevenue['ordersA'])-1)

# добавляем ось X
plt.axhline(y=0, color='black', linestyle='--')

plt.title('Относительное изменение кумулятивного среднего чека группы B к группе A')
plt.xlabel('Дата, день')
plt.ylabel('Отношение средних чеков')
plt.show()

Поскольку этот график является производным предыдущих, то и выводы по большей части аналогичные. Опять же здесь хорошо видно, что 19 августа был сильный всплеск (рост) за счет заказа 425 с аномально высокой стоимостью, а в последующие дни средний чек "B" также постепенно снижается.

###  График кумулятивной конверсии по группам.

In [ ]:
# считаем кумулятивную конверсию
cumulativeData['conversion'] = cumulativeData['orders']/cumulativeData['visitors']

# отделяем данные по группе A
cumulativeDataA = cumulativeData[cumulativeData['group']=='A']

# отделяем данные по группе B
cumulativeDataB = cumulativeData[cumulativeData['group']=='B']

# строим графики
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(cumulativeDataA['date'], cumulativeDataA['conversion'], label='A')
plt.plot(cumulativeDataB['date'], cumulativeDataB['conversion'], label='B')
plt.legend()
plt.title("Кумулятивная конверсия по группам")
plt.xlabel('Дата, день')
plt.ylabel('Конверсия')
 
# задаем масштаб осей
plt.axis(['2019-08-01', '2019-08-31', 0.028, 0.04])

plt.show()

Группы росли равномерно, но после 05-08-2019 конверсия группы В вырвалась вперёд и зафиксировалась, а конверсия группы А просела и также зафиксировалась.

### График относительного изменения кумулятивной конверсии группы B к группе A. 

In [ ]:
mergedCumulativeConversions = cumulativeDataA[['date','conversion']].merge(cumulativeDataB[['date','conversion']], left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

plt.figure(figsize=(15,5))
plt.grid()
plt.plot(mergedCumulativeConversions['date'], mergedCumulativeConversions['conversionB']/mergedCumulativeConversions['conversionA']-1, label="Относительный прирост конверсии группы B относительно группы A")
plt.legend()
plt.xlabel('Дата, день')
plt.ylabel('Значение прироста')

#Добавим пунктирную линию оси X чёрного цвета ('black') со значением оси Y = 0 методом plt.axhline(). 
plt.axhline(y=0, color='black', linestyle='--')
#И несколько линий оси X серого цвета ('grey')
plt.axhline(y=0.1, color='grey', linestyle='--')
plt.axhline(y=0.21, color='grey', linestyle='--')
plt.axhline(y=0.057, color='grey', linestyle='--')

plt.show()

В начале теста группа B проигрывала группе A, затем вырвалась резко вперёд 2 августа, а потом упала вновь 3 августа. Затем она начала расти до 15 августа (прирост 20%), после чего стала немного падать до 0.1, но все равно выигрывает по сравнению с группой А. Начиная с 6 августа группа "B" лидировала до конца периода; её конверсия росла до 15 августа и разница с "A" на максимуме составила 21%, затем пошло постепенное снижение отностельной конверсии до 10%-14%.

### Точечный график количества заказов по пользователям.

In [ ]:
ordersByUsers = (
    orders.drop(['group', 'revenue', 'date'], axis=1)
    .groupby('userId', as_index=False)
    .agg({'orderId': pd.Series.nunique})
)

ordersByUsers.columns = ['userId', 'orders']

ordersByUsers.sort_values(by='orders', ascending=False).head(10)

In [ ]:
# серия из чисел от 0 до количества наблюдений 
x_values = pd.Series(range(0, len(ordersByUsers)))

plt.figure(figsize=(15,5))
plt.grid()
plt.scatter(x_values, ordersByUsers['orders'], alpha=0.5) 

plt.title('Распределение числа заказов по пользователям')
plt.xlabel('Число пользователей, чел')
plt.ylabel('Число заказов, шт')
plt.show()

Больше всего пользователей сделали 1 заказ, но много пользователей с 2-4 заказами. Их точная доля не ясна — непонятно, считать их аномалиями или нет, нужно посмотреть процентили(95 и 99).

### 95-й и 99-й перцентили количества заказов на пользователя. Выбераем границу для определения аномальных пользователей.

In [ ]:
np.percentile(ordersByUsers['orders'], [95, 99])

Не более 5% пользователей совершали больше двух заказов. И не более 1% пользователей - больше четырех заказов.

###  Точечный график стоимостей заказов. 

In [ ]:
x_values = pd.Series(range(0, len(orders['revenue'])))

plt.figure(figsize=(15,5))
plt.grid()
plt.scatter(x_values, orders['revenue'], alpha=0.5) 

plt.title('Распределение стоимости заказов')
plt.xlabel('Число пользователей, чел')
plt.ylabel('Стоимость заказов, руб')
plt.show()

График не наглядный, сделаем меньше область анализа, хотя тут хорошо видно аномальный заказ 425 на 1,3 млн.рублей и еще один на ~200000 рублей. Предположение о том,что на резкий скачок в графиках выручки и среднего чека повлияли более дорогие заказы, подтвердилось.

In [ ]:
usual_orders = orders[orders['revenue']<=200000]['revenue']
xn_values = pd.Series(range(0,len(usual_orders)))
plt.figure(figsize=(15,5))
plt.grid()
plt.title('График стоимостей заказов')
plt.scatter(xn_values, usual_orders, alpha=0.5);

В основном заказы в районе стоимoсти 20000 рублей, иногда встречаются заказы в среднем до 100000 рублей.  
Чтобы найти более точную границу аномальных заказов рассчитаем процентили.

###   95-й и 99-й перцентили стоимости заказов.

In [ ]:
np.percentile(orders['revenue'], [95, 99])

Не более, чем у 5% заказов чек дороже 28000. И не больше, чем у 1% заказов чек дороже 58233 т.р.,  установим эту цифру в качестве границы для аномалий.

###   Cтатистическая значимость различий в конверсии между группами по «сырым» данным.

Соберем все данные по заказам и визитам, добавим кумулятивные показатели в одну таблицу. Для этого сделаем несколько таблиц и объединим их:

In [ ]:
visitorsADaily = visitors[visitors['group']=='A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']]. \
                                            agg({'date' : 'max', 'visitorsPerDateA' : 'sum'}), axis=1)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']

visitorsBDaily = visitors[visitors['group']=='B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']]. \
                                            agg({'date' : 'max', 'visitorsPerDateB' : 'sum'}), axis=1)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']


ordersADaily = orders[orders['group']=='A'][['date', 'orderId', 'userId', 'revenue']]\
    .groupby('date', as_index=False)\
    .agg({'orderId' : pd.Series.nunique, 'revenue' : 'sum'})
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: \
        ordersADaily[ordersADaily['date'] <= x['date']]\
            .agg({
                'date' : 'max',
                'ordersPerDateA' : 'sum',
                'revenuePerDateA' : 'sum'}), axis=1).sort_values(by=['date'])
ordersACummulative.columns = ['date', 'ordersCummulativeA', 'revenueCummulativeA']

ordersBDaily = orders[orders['group']=='B'][['date', 'orderId', 'userId', 'revenue']]\
    .groupby('date', as_index=False)\
    .agg({'orderId' : pd.Series.nunique, 'revenue' : 'sum'})
ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: \
        ordersBDaily[ordersBDaily['date'] <= x['date']]\
            .agg({
                'date' : 'max',
                'ordersPerDateB' : 'sum',
                'revenuePerDateB' : 'sum'}), axis=1).sort_values(by=['date'])
ordersBCummulative.columns = ['date', 'ordersCummulativeB', 'revenueCummulativeB']

In [ ]:
data = ordersADaily.merge(ordersBDaily, left_on='date', right_on='date', how='left')\
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')\
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')\
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')\
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')\
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')\
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
data.sample(3)

In [ ]:
ordersByUsersA = (
    orders[orders['group'] == 'A']
    .groupby('userId', as_index=False)
    .agg({'orderId': pd.Series.nunique})
)
ordersByUsersA.columns = ['userId', 'orders']


ordersByUsersB = (
    orders[orders['group'] == 'B']
    .groupby('userId', as_index=False)
    .agg({'orderId': pd.Series.nunique})
)
ordersByUsersB.columns = ['userId', 'orders'] 
ordersByUsersB.head()

sampleA = pd.concat(
    [
        ordersByUsersA['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)
sampleA.count()

sampleB = pd.concat(
    [
        ordersByUsersB['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)
sampleB.count()



display('p-value = {0:.3f}'.format(stats.mannwhitneyu(sampleA, sampleB, alternative = 'two-sided')[1]))
display("alpha =",0.05)
display('Относительный прирост конверсии группы B = {0:.3f}'.format(sampleB.mean()/sampleA.mean()-1)) 

 p-value 0.017 < 0.05. То есть, нулевую гипотезу, что статистически значимых различий в конверсии между группами нет, отвергаем. По «сырым» данным различия в конверсии групп A и B есть. Относительное преимущество группы "B" над "A" равен 13.8% .

### Статистическая значимость различий в среднем чеке заказа между группами по «сырым» данным.

In [ ]:
display('alpha =',0.05)
display('p-value = {0:.3f}'.format(stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'], alternative = 'two-sided')[1]))
display('Относительный прирост среднего чека группы B к среднему чеку группы А до очистки данных: = {0:.3f}'.format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1)) 

P-value больше 0.05, это говорит нам о том, что статистически значимых различий в среднем чеке заказа между группами по «сырым» данным нет, а различие между чеками 25.9% в пользу группы B.

### Cтатистическая значимость различий в конверсии между группами по «очищенным» данным.

В предыдущих пунктах мы приняли за аномальных пользователей тех, кто совершил 3 и больше заказов или совершил заказ дороже 58233.2 рублей. Сделаем срезы пользователей с числом заказов больше 2  и пользователей, совершивших заказы дороже 58233.2. Объединим их в одной таблице. Узнаем, сколько всего отличающихся пользователей методом shape().

In [ ]:
usersWithManyOrders = pd.concat([ordersByUsersA[ordersByUsersA['orders'] > 2]['userId'], ordersByUsersB[ordersByUsersB['orders'] > 2]['userId']], axis = 0)
usersWithExpensiveOrders = orders[orders['revenue'] > 58233.2]['userId']
abnormalUsers = pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis = 0).drop_duplicates().sort_values()
display(abnormalUsers.head(5)) 

In [ ]:
display('Всего отличающихся пользователей',len(abnormalUsers))

In [ ]:
sampleAFiltered = pd.concat(
    [
        ordersByUsersA[
            np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sampleBFiltered = pd.concat(
    [
        ordersByUsersB[
            np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
) 

За нулевую гипотезу возьмём что: "статистически значимых различий в конверсии между группами нет".  
Альтернативная гипотеза: статистически значимые различия есть.

In [ ]:
display('alpha =',0.05)
display('p-value = {0:.3f}'.format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered, alternative = 'two-sided')[1]))
display('Относительный прирост среднего чека группы B к среднему чеку группы А после очистки данных: = {0:.3f}'.format(sampleBFiltered.mean()/sampleAFiltered.mean()-1)) 

p-value 0.01 < 0.05. Нулевую гипотезу о том, что статистически значимых различий в конверсии между группами нет, отвергаем. Это значит, что и по «очищенным» данным различия в конверсии групп A и B есть. Относительный выигрыш группы "B" над "A" равен 17,4% (выше, чем с «сырыми» данными - 13.8%)

###  Статистическая значимость различий в среднем чеке заказа между группами по «очищенным» данным.

За нулевую гипотезу возьмём что: "статистически значимых различий в средних чеках между группами нет".  
Альтернативная гипотеза: статистически значимые различия есть.

In [ ]:
display("alpha =",0.05)
display("p-value = {0:.3f}".format(stats.mannwhitneyu(
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['userId'].isin(abnormalUsers)))]['revenue'],
    orders[np.logical_and(
        orders['group']=='B',
        np.logical_not(orders['userId'].isin(abnormalUsers)))]['revenue'], alternative = 'two-sided')[1]))

display("Относительный показатель 'B' и 'A' = {0:.3f}".format(
    orders[np.logical_and(orders['group']=='B',np.logical_not(orders['userId'].isin(abnormalUsers)))]['revenue'].mean()/
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['userId'].isin(abnormalUsers)))]['revenue'].mean() - 1)) 

P-value значительно больше 0.05. Значит, причин отвергать нулевую гипотезу и считать, что в среднем чеке есть различия,тоже нет, как и с "сырыми" данными. Однако средний чек группы B оказался незначительно (на 2%) ниже среднего чека группы A, в то время как по "сырым" данным, он был больше аж на ~26%, - и всё это из-за одного аномального заказа 425 на 1,3 млн.

#  Решение по результатам теста.

По результатам тестирования было выявлено, что по "сырым" и по "очищенным" данным различия в конверсиях групп A и B есть. Относительный выигрыш группы "B" над "A" равен 17,4% по "очищенным" и 13.8% по "сырым" данным.

Оказалось, что и по "сырым" и по "очищенным" данным различия в средних чеках групп A и B отстутствуют, несмотря на то, что средний чек группы B оказался незначительно (на 2%) ниже среднего чека группы A по "очищенным" данным, в то время как по "сырым" данным, он был больше где-то на 26%, всё это из-за одного аномального заказа 425 на 1,3 млн. Аномальный заказ убрали из выборки и чеки сравнялись.

Здесь можно принять решение остановить тестирование и констатировать преимущество группы B, так как группа B значительно  эффективнее группы A по конверсии, хотя по среднему чеку группы примерно равны. 